<a href="https://colab.research.google.com/github/BenjamimOliveira/CN_TP1/blob/main/script/Script2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP1 - CN


## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import tensorflow.keras.optimizers as optimizers
import time
import zipfile
import random

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Load data sources

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

start_time = time.time()

# Copy data (.zip file)
!cp -r 'drive/MyDrive/UMinho/CN/dataset.zip' 'dataset.zip'

# Extract .zip file
with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset_extr')

print("%s segundos" % ((time.time() - start_time)))

Mounted at /content/drive
33.83084487915039 minutos


## Data Loading

In [3]:
train_dir = '/content/dataset_extr/dataset/train'
test_dir = '/content/dataset_extr/dataset/test'
valid_dir = '/content/dataset_extr/dataset/valid'

imgDataGenerator = False

if imgDataGenerator:
  train_dtgen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
  test_dtgen = ImageDataGenerator(rescale = 1./255)
  valid_dtgen = ImageDataGenerator(rescale = 1./255)

  train_ds = train_dtgen.flow_from_directory(train_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32)
  test_ds = test_dtgen.flow_from_directory(test_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32)
  valid_ds = valid_dtgen.flow_from_directory(valid_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32)
else:
  train_ds = tf.keras.preprocessing.image_dataset_from_directory(
      train_dir,
      seed=123,
      image_size=(64, 64),
      batch_size=32)
  test_ds = tf.keras.preprocessing.image_dataset_from_directory(
      test_dir,
      seed=123,
      image_size=(64, 64),
      batch_size=32)
  valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
      valid_dir,
      seed=123,
      image_size=(64, 64),
      batch_size=32)
  
  class_names = train_ds.class_names

  AUTOTUNE = tf.data.AUTOTUNE

  train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)
  test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 35215 files belonging to 250 classes.
Found 1250 files belonging to 250 classes.
Found 1250 files belonging to 250 classes.


## Algoritmo Genético

In [4]:
##
# Formato do gene [nParesConv2DMaxPooling(entre 1 e 3), learningRate, Momentum, Nesterov, batchSize, Epochs, Accuracy]
#
##



### Gene Aleatório

In [6]:
def random_element():
  gene = []
  
  # -- nPares Conv2D/MaxPooling
  gene.append(random.randint(1,3))

  # -- Learning rate
  gene.append(random.randint(0,4))

  # -- Momentum
  gene.append(random.randint(0, 9)/10)

  # -- Nesterov
  gene.append(random.randint(0,1))

  # -- BatchSize
  gene.append(random.randint(batchSize[0],batchSize[1]))

  # -- Epochs
  gene.append(random.randint(epochs[0], epochs[1]))

  # -- Accuracy
  gene.append(-1)

  return gene

### População Aleatória/Inicial

In [7]:
def random_pool_generator(poolSize):
  pool = []

  for x in range(poolSize):
    pool.append(random_element())

  return pool

### Selecionar "pais" da próxima geração

In [38]:
def select_mating_pool(poolSize, pool):
  top = round(poolSize/2)
  def orderBy(a):
    return a[6]

  pool.sort(reverse=True, key=orderBy)
  return pool[0:top]
  

### Crossover entre 2 elementos

In [89]:
def mix2elements(elemento1, elemento2):
  # tamanho do gene
  size = len(elemento1) - 1
  # elementos que irão ser cruzados
  crossElements = []
  for x in range(3):
    rand = random.randint(0, size-1)
    while (rand in crossElements):
      rand = random.randint(0, size-1)
    crossElements.append(rand)

  elementoFilho = []
  aux = 0

  while aux < size:
    if aux in crossElements:
      elementoFilho.append(elemento1[aux])
    else:
      elementoFilho.append(elemento2[aux])
    aux += 1
  # indica que ainda não foi treinado  
  elementoFilho.append(-1)
  return elementoFilho

### Selecionar que elementos vão fazer crossover

In [91]:
def crossover(poolSize, pool):
  currentPool = poolSize

  # mistura o primeiro com o último
  pool.append(mix2elements(pool[0], pool[-1]))

  aux = 0
  for i in range(int(round(poolSize/2))-1):
    pool.append(mix2elements(pool[aux], pool[aux+1]))
  return pool

### Mutação de um elemento

In [139]:
def mutate(gene, numberOfMutations):
  size = len(elemento1) - 1
  mut = []
  for x in range(numberOfMutations):
    rand = random.randint(0, size-1)
    while (rand in mut):
      rand = random.randint(0, size-1)
    mut.append(rand)
  
  element = random_element()
  for x in mut:
    gene[x] = element[x]
  
  return gene
  

### Selecionar elementos que irão sofrer mutações

In [140]:
def mutation(pool, chance):
  chance = int(round(chance * 100))
  count = 0
  for gene in pool:
    rand = random.randint(0,100)
    # Decide se um gene vai ou não ser mutado
    if gene[:-1] == -1:
      if rand < chance:
        gene = mutate(gene, random.randint(1,2))
      
  return pool

### Gerador de CNN (segundo o gene)

In [12]:
def model_generator(nPares):
  mod = []
  mod.append(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(64, 64, 3)))
  if nPares >= 1: 
    mod.append(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
    mod.append(layers.MaxPooling2D())
  if nPares >= 2:
    mod.append(layers.Conv2D(32, 3, padding='same', activation='relu'))
    mod.append(layers.MaxPooling2D())
  if nPares >= 3:
    mod.append(layers.Conv2D(64, 3, padding='same', activation='relu'))
    mod.append(layers.MaxPooling2D())
  mod.append(layers.Flatten())
  mod.append(layers.Dense(128, activation='relu'))
  mod.append(layers.Dense(num_classes))
  model = Sequential(mod)
  return model

### Conversor de gene em acurácia (treino da cnn com o gene)

In [16]:
def gene_converter(gene):
  # -- MODELO
  model = model_generator(gene[0])
  # -- COMPILAR MODELO
  opt = optimizers.SGD(learning_rate=0.01, momentum=0.9)
  # Experimentar loss='sparse_categorical_crossentropy'
  if imgDataGenerator:
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  else:
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  model.compile(optimizer='adam',
                loss=loss,
                metrics=['accuracy'])
  # -- TREINAR MODELO
  epochs=1
  start_time = time.time()
  with tf.device('/device:GPU:0'):
    history = model.fit(
      train_ds,
      validation_data=valid_ds,
      epochs=epochs
    )
  # -- AVALIAR MODELO
  results = model.evaluate(test_ds)
  return results[1]

### Algoritmo Final

In [142]:
# --------- Parametros chave ---------
learningRate = [1, 0.1, 0.01, 0.001, 0.0001]
batchSize = [8, 128]
epochs = [4, 20]
num_classes = 250
poolSize = 4
mutationChance = 1
geracoes = 2
# ------------------------------------

pool = random_pool_generator(poolSize)
print("Inicial: ",pool)

maximiza = []

# treina a geração
for gene in pool:
  # -- se ainda não tiver sido treinado numa geração passada então treina
  if gene[6] == -1:
    gene[6] = gene_converter(gene=gene)
  
# seleciona metade do poolsize para se reproduzir
pool = select_mating_pool(poolSize, pool)
pool = crossover(poolSize, pool)
print("Crossover: ",pool)
pool = mutation(pool, mutationChance)
print("Mutação: ",pool)

Inicial:  [[2, 2, 0.8, 1, 94, 11, -1], [1, 2, 0.1, 0, 70, 7, -1], [3, 3, 0.2, 1, 116, 17, -1], [3, 2, 0.7, 1, 120, 6, -1]]
1101/1101 [==============================] - 5s 4ms/step - loss: 5.0284 - accuracy: 0.0547 - val_loss: 2.9109 - val_accuracy: 0.3088
0.07537527879079182 minutos
1101/1101 [==============================] - 4s 3ms/step - loss: 5.0562 - accuracy: 0.0449 - val_loss: 3.4832 - val_accuracy: 0.2112
0.06578121582667033 minutos
1101/1101 [==============================] - 5s 4ms/step - loss: 5.1326 - accuracy: 0.0359 - val_loss: 3.3713 - val_accuracy: 0.2328
0.07725036144256592 minutos
1101/1101 [==============================] - 5s 4ms/step - loss: 5.1515 - accuracy: 0.0367 - val_loss: 3.3265 - val_accuracy: 0.2368
0.07832053502400717 minutos
40/40 [==============================] - 0s 2ms/step - loss: 3.3158 - accuracy: 0.2432
Elemento:
<class 'list'>
[2, 2, 0.8, 1, 94, 11, 0.3407999873161316]
Elemento:
<class 'list'>
[2, 2, 0.8, 1, 94, 11, 0.3407999873161316]
Crossover:

In [136]:
#for x in pool:
#  print(x)
#print(len(pool[0]))
#print(random.randint(0,5))
elemento1 = [1,1,1,1,1,1,-1]
#elemento2 = [2,2,2,2,2,2,-1]
#print(len(elemento1)-1)

#pool = random_pool_generator(poolSize)
#print(pool)
#pool = mutation(pool, 0.9)

gene = mutate(elemento1, 2)
print(gene)

random:  [1, 1, 0.8, 0, 87, 8, -1]
[1, 1, 1, 0, 87, 1, -1]
